# 🎓 벡터와 행렬: 딥러닝 실습

## 1. 벡터란?
- **벡터**는 여러 숫자를 한 줄로 나열한 **리스트**입니다.
- 예를 들어 `[3, 4]`는 2차원 벡터(숫자 2개)예요.
- 아래 코드 셀을 실행해 벡터를 만들어 봅시다.

In [ ]:
import numpy as np
v = np.array([3, 4])
print('벡터 v:', v)
print('차원(숫자 개수):', v.shape)

## 2. 벡터 연산: 덧셈과 뺄셈
같은 위치(차원)의 숫자끼리 더하거나 빼면 됩니다.

In [ ]:
v1 = np.array([1, 2])
v2 = np.array([3, 4])
print('v1 + v2 =', v1 + v2)
print('v1 - v2 =', v1 - v2)

## 3. 내적(Dot Product)
각 위치 숫자를 곱한 뒤 모두 더합니다. 방향이 비슷하면 큰 값이 나와요.

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print('a · b =', a @ b)

## 4. 벡터의 길이(노름, Norm)
- **L1 노름**: 절댓값을 모두 더함
- **L2 노름**: 피타고라스 거리(가장 많이 사용)
- **무한 노름**: 가장 큰 숫자 하나만 보기

In [ ]:
from numpy.linalg import norm
vec = np.array([3, 4])
print('L1 노름:', norm(vec, 1))
print('L2 노름:', norm(vec))
print('무한 노름:', norm(vec, np.inf))

## 5. 행렬(Matrix)란?
- 행(row)과 열(column)로 숫자를 정리한 **표**입니다.
- 예: 아래 2×3 행렬을 만들어 봅시다.

In [ ]:
M = np.array([[1, 2, 3], [4, 5, 6]])
print('행렬 M:\n', M)
print('모양(shape):', M.shape)

### 5‑1. 이미지도 행렬!
아래는 3×3 흑백 이미지(밝기 값) 예시입니다.

In [ ]:
img = np.array([[0, 255, 0],
                 [255, 255, 255],
                 [0, 255, 0]])
print(img)

## 6. 딥러닝 적용 예시

In [ ]:
#!git clone https://github.com/simsimee/HKNU_lecture.git #하지 않은 경우 #을 지우고 실행!

In [ ]:
!pip install scikit-learn sentence-transformers torch

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import os

image_paths = sorted(glob('/content/HKNU_lecture/데이터/예시_이미지/*'))

print('이미지 개수:', len(image_paths))

In [ ]:
# 썸네일 표시
plt.figure(figsize=(3 * len(image_paths), 3))
for i, p in enumerate(image_paths, 1):
    plt.subplot(1, len(image_paths), i)
    img = Image.open(p)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
### 유사한 이미지 찾기

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('clip-ViT-B-32')

In [ ]:
# 이미지 임베딩 (텐서 반환)
image_embs = model.encode(
    [Image.open(p) for p in image_paths],
    convert_to_tensor=True,
    show_progress_bar=True
)
print("image_embs.shape:", image_embs.shape)

In [ ]:
# 텍스트 기반 이미지 검색
query_text = 'cute dog'

query_emb = model.encode(query_text, convert_to_tensor=True)
cos_scores = util.cos_sim(query_emb, image_embs)[0]

top_k = torch.topk(cos_scores, k=min(3, len(image_paths)))

print("쿼리:", query_text, "\n")

plt.figure(figsize=(4 * top_k.indices.size(0), 4))
for idx_rank, idx in enumerate(top_k.indices, 1):
    plt.subplot(1, top_k.indices.size(0), idx_rank)
    plt.imshow(Image.open(image_paths[idx]))
    plt.title(f'score: {cos_scores[idx]:.3f}')
    plt.axis('off')
plt.show()

In [ ]:
import numpy as np
#이미지 기반 유사 이미지 검색
query_idx = 0  # 기준이 될 이미지 인덱스
query_emb_img = image_embs[query_idx]

cos_scores_img = util.cos_sim(query_emb_img, image_embs)[0]
# 자기 자신을 제외하고 상위 3개
top_indices = cos_scores_img.argsort(descending=True).cpu().numpy()[1:4]

plt.figure(figsize=(4 * (len(top_indices) + 1), 4))

# 원본
plt.subplot(1, len(top_indices) + 1, 1)
plt.imshow(Image.open(image_paths[query_idx]))
plt.title('Query')
plt.axis('off')

# 유사 이미지
for i, idx in enumerate(top_indices, 2):
    plt.subplot(1, len(top_indices) + 1, i)
    plt.imshow(Image.open(image_paths[idx]))
    plt.title(f'score: {cos_scores_img[idx]:.3f}')
    plt.axis('off')

plt.show()

In [ ]:
## 유사한 문장 찾기

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
sent1 = '고양이는 집 안을 돌아다니고 있다.'
sent2 = '고양이가 실외를 배회하고 있다.'
embeddings = model.encode([sent1, sent2])

pca = PCA(n_components=2)
pts = pca.fit_transform(embeddings)
plt.scatter(pts[:,0], pts[:,1])
for i, txt in enumerate(['sent1', 'sent2']):
    plt.annotate(txt, (pts[i,0], pts[i,1]))
plt.title('Sentence Embeddings (PCA)')
plt.xlabel('PC1'); plt.ylabel('PC2'); plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 계산
cos_sim = cosine_similarity([embeddings[0]], [embeddings[1]])
print(f"코사인 유사도: {cos_sim[0][0]:.4f}")

## 7. 정리
- 벡터와 행렬은 AI 계산의 기본 재료입니다.
- 이 노트북에서 본 간단한 연산이 거대한 딥러닝 모델에서도 반복됩니다.

🎉 수고했습니다!